In [1]:
# importing necessary libraries

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import sqlalchemy
import json

In [2]:
# Retrive the bus details

driver = webdriver.Chrome()

Bus_names = []
Bus_types = []
Start_Time = []
End_Time = []
Ratings = []
Total_Duration = []
Prices = []
Seats_Available = []
Route_names = []
Route_link = []

def right_button():
    try:
        right_button = driver.find_elements(By.XPATH, "//i[@class = 'icon icon-right']")
        right_button[0].click()
        time.sleep(5)
    except:
        pass

# Loop through route links and names

for i, r in df.iterrows():
    link = r["Route_Link"]
    routes = r["Route_Name"]

    driver.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        # time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        for x in clicks:
            x.click()
            time.sleep(1)
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")

    try:
        rating = driver.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")

    except:
        continue
    price = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_link.append(link)
        Route_names.append(routes)

    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)

    for start_time_elem in start_time:
        Start_Time.append(start_time_elem.text)

    for end_time_elem in end_time:
        End_Time.append(end_time_elem.text)

    for total_duration_elem in total_duration:
        Total_Duration.append(total_duration_elem.text)

    for ratings in rating:
        Ratings.append(ratings.text)

    for price_elem in price:
        Prices.append(price_elem.text)

    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)
        
print("Successfully Completed")

NameError: name 'df' is not defined

In [ ]:
driver.quit()

In [ ]:
# From list to convert data frame

data = {
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Start_time': Start_Time,
    'End_time': End_Time,
    'Total_duration': Total_Duration,
    'Price': Prices,
    "Seats_Available":Seats_Available,
    "Ratings":Ratings,
    'Route_link': Route_link,
    'Route_name': Route_names
}

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
df.to_csv(r'C:\IT\learn\projects\redbus_webscraping\csv_files\bus_details.csv', index = False)

### Exploratory Data Analysis and Data Cleansing

In [ ]:
import numpy as np

In [ ]:
df = pd.read_csv(r'C:\IT\learn\projects\redbus_webscraping\csv_files\bus_details.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#Price have INR and it is a non numerical

df['Price'][0] # printing one element of rating

In [ ]:
df["Price"] = df["Price"].str.replace("INR ","") # Removing the INR 

In [ ]:
df["Price"]=df["Price"].astype(float) # converting string to float
df["Price"].fillna(0) #if null values it will will with zero Treating null values

In [ ]:
df['Ratings']

In [ ]:
df ['Ratings'] = df['Ratings'].str.replace("New","")

Use excel filter to analyse the csv extracted file. Some of the ratings have formate of 5.1 30 

format float int

In [ ]:
df['Ratings'] = df['Ratings'].str.strip()

In [ ]:
df.head()

In [ ]:
df.to_csv("checking.csv")

In [ ]:
df["Ratings"]=df["Ratings"].str.split().str[0]

In [ ]:
df.head()

In [ ]:
# try to convert to numeric and if fails fill with NaN (not a number)

df["Ratings"] = pd.to_numeric(df["Ratings"], errors='coerce')

In [ ]:
df["Ratings"].fillna(0, inplace=True) 

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.replace({np.nan: None})

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna(subset=['Bus_name'])

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna(subset=['Bus_type'])

In [ ]:
df = df.dropna(subset=['Price'])

In [ ]:
df = df.dropna(subset=['Seats_Available'])

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.to_csv("C:\IT\learn\projects\redbus_webscraping\checks.csv")

In [ ]:
df = df.fillna({'Bus_type':"unknown",'Seats_Available':0})

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna(subset=['Price'])

In [ ]:
df.isnull().sum()